# SNAP Gap in Los Angeles County: Spatial Patterns and Predictors
#### Meaghan Woody


## Urban Planning 213 Final Project 

#### **Background:**

One of the most urgent priorities for anti-hunger nonprofit organizations and CalFresh agencies in California is targeted outreach to eligible but unenrolled households. However, few tools exist to systematically identify communities where enrollment is low, even though need is high. “SNAP Gap” refers to the proportion of those who are eligible but not enrolled in Federally funded Supplemental Nutrition Assistance Program (SNAP). California ranks significantly lower than half the US (81% in 2022, 67% in 2020), yet has the highest number of SNAP recipients (USDA, 2022). In 2020, it was reported there is an estimated 280,000 LA County households who remain eligible for but are not receiving CalFresh. In general, to get CalFresh benefits, the household's gross income must be less than the gross-income eligibility standard 130% of the Federal Poverty Level (FPL).

Understanding neighborhood-level aspects of SNAP participation can greatly inform outreach efforts. Mapping tools can help identify and share opportunities to maximize efforts by targeting local areas with the lowest participation rates. Low SNAP participation rates in California cannot be attributed to a single factor but rather a multitude of factors that span across policy, politics, administration, demographics, and cultural attitudes, therefore this project aims to identify predictors of SNAP gap through an exploratory analysis. 

#### **Research aims:** 

1. Map the distribution of SNAP gap and food insecurity in Los Angeles County in varying geographic units.
2. Conduct a spatial cluster analysis of SNAP gap rates
3. Identify predictors that explain SNAP gap using machine learning models trained on American Community Survey (ACS) variables.

#### **References:**

1. U.S. Department of Agriculture, Food and Nutrition Service. (2025, April 18). Reaching those in need: Estimates of state SNAP participation rates in 2022. https://www.fns.usda.gov/research/snap/state-participation-rates/2022
2. Los Angeles Regional Food Bank. (n.d.). Closing the SNAP Gap. Retrieved June 11, 2025, from https://www.lafoodbank.org/stories/closing-the-snap-gap/#:~:text=The%20gap%20during%202019%20was,pandemic%2C%20a%20gap%20still%20remains. https://www.lafoodbank.org/closing-the-snap-gap/.

#### **Workflow of notebooks:**

`0_Project intro & Data prep`: Data cleaning for all the ACS variables

`1_Mapping SNAP Rates`: Aims 1 and 2

`2_Machine Learning`: Aim 3

### 1) Access Census Data using the Census Bureau API

The table below shows all the variables needed to be pulled from the American Community Survey (2023) through the Census Bureau API.

The variable names can be accessed at here: https://api.census.gov/data/2023/acs/acs5/variables.html

In general, to get CalFresh benefits, the household's gross income must be less than the gross-income eligibility standard 130% of the Federal Poverty Level (FPL)

| ACS Variable(s) | ACS Description | Denominator | New Variable Name |
|------------------|-----------------|-------------|-------------------|
|**==SNAP Gap Numerator==**|
| B22003_006E | Household did not receive Food Stamps/SNAP in the past 12 months:!!Income in the past 12 months below poverty level | 
|**==SNAP Gap Denominator==**|
| C17002_002E
| C17002_003E
| C17002_004E
| C17002_005E
|**==Demographics==**|
| B05002_021E | Foreign-born: Not a U.S. citizen | B05002_013E | %_foreign_born |
| B05002_026E | Not a U.S. citizen from Latin America | B05002_021E | %_noncitizen_latam |
| B01001_020E to B01001_025E (males), B01001_044E to B01001_049E (females) | Population aged 65+ | B01001_002E + B01001_026E | %_seniors |
| B03003_003E | Hispanic or Latino | B03003_001E | %_hispanic |
| B02001_003E | Black or African American alone | B02001_001E | %_black |
| B02001_005E | Asian alone | B02001_001E | %_asian |
|**==Socioeconomics==**
| B17001_002E | Income below poverty level | B17001_001E | %_poverty |
| B19013_001E | Median household income | N/A | median_income |
| B23025_004E | Employed (age 16+) | B23025_002E | %_employed |
| B14001_008E | Enrolled in college, undergraduate years | B14001_002E | %_college_undergrad |
| B14001_009E | Enrolled in graduate or professional school | B14001_002E | %_grad_students |
|**==Household==**
| B25010_001E | Average household size | N/A | avg_household_size |
| B25003_003E | Renter occupied | B25003_001E | %_renters |
| B25070_007E to B25070_010E | Rent burdened (30% or more of income) | B25070_001E | %_rent_burdened |
| B28002_013E | No Internet access | B28002_001E | %_no_internet |



In [39]:
import json
import requests
import pandas as pd

r = requests.get('https://api.census.gov/data/2023/acs/acs5?get=B22003_001E&B17017_002E&B22003_002E&B22003_003E&B22003_005E&B22003_006E&for=tract:*&in=state:06%20county:037')
censusdata = r.json()
census_snap = pd.DataFrame(censusdata[1:], columns=censusdata[0])
census_snap.head()

,B22003_001E,B17017_002E,B22003_002E,B22003_003E,B22003_005E,B22003_006E,state,county,tract
0,1558,247,283,103,1275,144,06,037,101110
1,1407,69,160,42,1247,27,06,037,101122
2,1357,215,324,149,1033,66,06,037,101220
3,1483,275,433,116,1050,159,06,037,101221
4,948,287,284,169,664,118,06,037,101222


In [40]:
# Rename the variables 
census_snap = census_snap.rename(columns={
    'B22003_001E': 'total_hh',
    'B17017_002E': 'total_pov',
    'B22003_002E': 'snap_tot_hh',
    'B22003_003E': 'snap_pov_hh',
    'B22003_005E': 'nosnap_tot_hh',
    'B22003_006E': 'nosnap_pov_hh'
    # Foreign-born & non-citizen
    "B05002_013E", "B05002_021E", "B05002_026E",
    # Seniors
    "B01001_020E", "B01001_021E", "B01001_022E", "B01001_023E", "B01001_024E", "B01001_025E",
    "B01001_044E", "B01001_045E", "B01001_046E", "B01001_047E", "B01001_048E", "B01001_049E",
    "B01001_002E", "B01001_026E",
    # Race/ethnicity
    "B03003_003E", "B03003_001E",
    "B02001_003E", "B02001_005E", "B02001_001E",
    # Poverty
    "B17001_002E", "B17001_001E",
    # Income
    "B19013_001E",
    # Employment
    "B23025_004E", "B23025_002E",
    # Students
    "B14001_008E", "B14001_009E", "B14001_002E",
    # Household
    "B25010_001E",
    "B25003_003E", "B25003_001E",
    "B25070_007E", "B25070_008E", "B25070_009E", "B25070_010E", "B25070_001E",
    "B28002_013E", "B28002_001E",
    #SNAP
    "B22003_005E"
]

# Step 2: Join variables
var_str = ",".join(variables)

# Step 3: Construct URL
url = f"https://api.census.gov/data/2023/acs/acs5?get={var_str}&for=tract:*&in=state:06%20county:037"

# Step 4: Make request
response = requests.get(url)

# Step 5: Parse JSON from response
data = response.json()

# Step 6: Create DataFrame
predictors = pd.DataFrame(data[1:], columns=data[0])

# Step 7: Convert numeric columns
numeric_cols = [col for col in predictors.columns if col not in ['state', 'county', 'tract']]
predictors[numeric_cols] = predictors[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Step 8: Preview
predictors.head()

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 2 (988756585.py, line 33)